In [1]:
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import requests
import json
import re
import os

/Users/tarosh/opt/anaconda3/envs/betterSectionTally/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from backend.catalog import Catalog
from backend.sectiontally import SectionTally

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
tally = SectionTally(term='202520', subj='ALL', dept='ALL', attr='ALL')

100%|██████████| 4695/4695 [00:00<00:00, 9808.00it/s]


In [4]:
catalog = Catalog(tally=tally, catalog_json='./catalog.json', force_download=True)

100%|██████████| 149/149 [07:00<00:00,  2.82s/it]


AttributeError: 'Catalog' object has no attribute 'data'

In [31]:
def extract_preqs(soup) -> str:
    """
        extracts prerequisites from Rowan's detailed course information website
        
        Args:
            soup: BeautifulSoup of Rowan course information HTML
        Returns:
            preqs: string
    """
    preq_head = soup.find('span', 'fieldlabeltext', string=re.compile('Prerequisites', re.IGNORECASE))
    if preq_head == None:
        return None

    assert (preq_head.next_siblings != None), "nothing found after 'Prerequisites: '"
        
    res = []
    prev_tag = ''
    for sibling in preq_head.next_siblings:
        if sibling == '\n':
            continue

        if (sibling.name == 'br') and (prev_tag == 'br'):
            break

        prev_tag = sibling.name

        s = sibling.string
        if s != None:
            res.append(s)

    return ''.join(res)

In [151]:
text = catalog.data['CS 04400']['preqs']
words_to_remove = ['Undergraduate level', 'Graduate level', 'Minimum Grade of']
pattern = r'\b(?:' + '|'.join(words_to_remove) + r')\b'
out = re.sub(pattern, ' ', text)
out = ' '.join(re.sub(r'([A-Z][+|-])', r'{min \1}', out).split())
preqs = re.findall(r'(\w{2,5} \d{5})', out)

In [152]:
for preq in preqs:
    if preq in catalog.data:
        title = catalog.data[preq]['title']
        out = out.replace(preq, title)
out

'( CS 01205 {min D-} or CS 04215 - Computer Laboratory Techniques {min D-}) and CS 07340 - Design and Analysis of Algorithms {min D-} and ( CS 07321 {min D-} or CS 04321 - Software Engineering I {min D-})'